In [1]:
import json
import matplotlib.pyplot as plt
import matplotlib
from collections import defaultdict

output_file = '../data/cadec/output.json'
gt_file = '../data/cadec/test.json'
train_file = '../data/cadec/train.json'
with open(output_file,'r') as fp:
    output = json.load(fp)
with open(gt_file, 'r') as fp:
    gt = json.load(fp)
with open(train_file, 'r') as fp:
    train = json.load(fp)

In [17]:
# discontinuous entities
# l is the index list
def isDiscontinuous(l):
    lastIndex = l[0]
    for idx in range(1, len(l)):
        if l[idx] != (lastIndex + 1):
            return True
        lastIndex = l[idx]

# ner_list is the ner map in the groundtruth
def overlapCount(ner_list):
    position_set = [0 for i in range(200)]
    overlap_cnt = 0
    for ner in ner_list:
        ner_index = ner['index']
        for idx in ner_index:
            if position_set[idx] == 0:
                position_set[idx] = 1
            else:
                overlap_cnt += 1
                break
    return overlap_cnt

# ner: the list of index for an entity
# ner_list: the list of above (list + type)
# check whether ner is an overlapped ner in the ner_list
def isOverlap(target_ner, ner_list):
    position_set = [0 for i in range(200)]
    for ner in ner_list:
        if target_ner == ner['index']:
            continue
        else:
            ner_index = ner['index']
            for idx in ner_index:
                position_set[idx] = 1
    for idx in target_ner:
        if position_set[idx] == 1:
            return True
    return False


In [3]:
gt[0]

{'sentence': ['Brain', 'fog', 'and', 'decreased', 'cognitive', 'skills', '.'],
 'ner': [{'index': [0, 1], 'type': 'ADR'},
  {'index': [3, 4, 5], 'type': 'ADR'}]}

In [5]:
sentence

['Lowered', 'my', 'Cholesterol', 'from', '255', 'to', '173', '.']

In [6]:
ners

[]

In [9]:
output[0]['entity']

[{'text': ['decreased', 'cognitive', 'skills'], 'type': 'adr'},
 {'text': ['Brain', 'fog'], 'type': 'adr'}]

In [26]:
total_discontinuous = 0
total_discontinuous_correct = 0
total_overlap = 0
total_overlap_correct = 0

for sentence_idx in range(len(gt)):
    sentence = gt[sentence_idx]['sentence']
    ners = gt[sentence_idx]['ner']
    for ner in ners:
        ner_precision = False
        # check whether in the test set out
        tokens = [sentence[i] for i in ner['index']]
        tokens_type = {'text':tokens, 'type':'adr'}

        if tokens_type in output[sentence_idx]['entity']:
            ner_precision = True

        # discontinuous
        if isDiscontinuous(ner['index']):
            total_discontinuous += 1
            if ner_precision:
                total_discontinuous_correct += 1
        
        # print(ner)
        # print(ners)
        if isOverlap(ner['index'], ners):
            total_overlap += 1
            if ner_precision:
                total_overlap_correct += 1
            else:
                print(sentence)

print("in total", total_discontinuous, "discontinuous entities, precision", total_discontinuous_correct/total_discontinuous)
print("in total", total_overlap, "overlap entities, precision:",total_overlap_correct/total_overlap)

['Did', 'help', 'lower', 'my', 'bad', 'cholesterol', 'but', 'caused', 'me', 'to', 'crave', 'sweets', 'and', 'carbohydrates', '.']
['Did', 'help', 'lower', 'my', 'bad', 'cholesterol', 'but', 'caused', 'me', 'to', 'crave', 'sweets', 'and', 'carbohydrates', '.']
['Severe', 'muscle', 'cramps', ',', 'charlie', 'horse', 'in', 'my', 'calves', 'and', 'feet', 'so', 'bad', 'at', 'night', 'that', 'it', 'brings', 'me', 'out', 'of', 'bed', 'screaming', '.']
['Shoulder', 'pain', ',', 'tendonitis', 'in', 'my', 'shoulders', 'and', 'elbows', '.']
['Reduced', 'cholesterol', 'levels', 'to', 'normal', 'BUT', 'since', 'taking', 'lipitor', 'I', "'", 've', 'developed', 'horrible', 'pain', 'in', 'both', 'shoulders', 'and', 'left', 'knee', '.']
['my', 'partner', 'is', 'currently', 'taking', 'this', 'medication', 'liptor', ',', 'he', 'also', 'takes', 'high', 'blood', 'pressure', 'medication', ',', 'my', 'reason', 'for', 'initally', 'looking', 'for', 'side', 'effects', 'of', 'liptor', 'was', 'because', 'of', 'my

In [19]:
print(total_discontinuous)

94


In [20]:
print(total_discontinuous_correct)

33


In [21]:
print(total_overlap)

131


In [22]:
print(total_overlap_correct)

70
